In [2]:
using Random, Statistics
using FFTW

In [3]:
Nx, Ny = 8, 8
R = rand(MersenneTwister(0), Nx, Ny)
R = R .- mean(R)

8×8 Array{Float64,2}:
  0.330877  -0.130942    0.0831171  …   0.45892   -0.308655    0.0555719
  0.417586   0.480446    0.375508       0.308348  -0.180625   -0.0158876
 -0.328204   0.0930413   0.475029      -0.368448  -0.296363    0.109604 
 -0.315441   0.046519    0.27492       -0.378501   0.380811    0.298576 
 -0.21389   -0.232734   -0.0234665     -0.413216   0.162151   -0.443648 
 -0.289294   0.417276   -0.430403   …   0.283904   0.0939422  -0.0100088
 -0.450469  -0.325734   -0.139641      -0.387948   0.139925    0.172481 
 -0.424501   0.162678    0.274832       0.345305   0.242234    0.349486 

In [4]:
R2 = copy(R)
R2 .= cat(R2[1:2:Nx, :], R2[Nx:-2:2, :]; dims=1)
R2 .= cat(R2[:, 1:2:Ny], R2[:, Ny:-2:2]; dims=2)

8×8 Array{Float64,2}:
  0.330877   0.0831171  -0.192695    …   0.45892   -0.449629  -0.130942 
 -0.328204   0.475029   -0.373118       -0.368448  -0.425924   0.0930413
 -0.21389   -0.0234665   0.309153       -0.413216   0.112526  -0.232734 
 -0.450469  -0.139641   -0.00810973     -0.387948   0.345347  -0.325734 
 -0.424501   0.274832    0.406429        0.345305   0.421942   0.162678 
 -0.289294  -0.430403   -0.457426    …   0.283904  -0.357026   0.417276 
 -0.315441   0.27492     0.274299       -0.378501  -0.336134   0.046519 
  0.417586   0.375508    0.23008         0.308348  -0.224786   0.480446 

In [5]:
@inline W(M, k) = exp(-2im*π*k/M)

ω_4Nx_kx⁺ = reshape(W.(4Nx, 0:Nx-1), Nx, 1)
ω_4Nx_kx⁻ = reshape(W.(4Nx, 0:-1:-(Nx-1)), Nx, 1)
ω_4Ny_ky⁺ = reshape(W.(4Ny, 0:Ny-1), 1, Ny)
ω_4Ny_ky⁻ = reshape(W.(4Ny, 0:-1:-(Ny-1)), 1, Ny)

R3 = fft(R2)

r_inds = [1, collect(Nx:-1:2)...]
R3⁻ = view(R3, r_inds, 1:Ny)

R4 = zeros(Nx, Ny)
@. R4 = 2 * real(ω_4Ny_ky⁺ * (ω_4Nx_kx⁺ * R3 + ω_4Nx_kx⁻ * R3⁻))

8×8 Array{Float64,2}:
  -1.33227e-14  -4.64441   -0.109857  …  -7.91735  -0.0702232   5.77614
  -0.818832     10.7835     8.20803      -4.48948  11.4555      4.42911
  11.8256       -0.275144  -1.08929      -1.47803   8.57057    -7.34959
  -8.78344       0.752585   3.21672       2.83585  -0.234098   -6.27776
   6.37467      -3.86379   -5.32137       2.05172  -0.350748    5.54903
  -9.31376      -6.25196    4.16038   …   2.19562   0.708704    1.78961
  -6.3583       -0.814365   5.45273      -3.69907   4.16756    -6.03769
 -11.1435        2.22139   -5.64859       6.05728   0.744963    2.47283

In [6]:
R4 ≈ FFTW.r2r(R, FFTW.REDFT10)

true

In [12]:
ω_4Nx_kx⁺ = reshape(W.(4Nx, 0:Nx-1), Nx, 1)
ω_4Nx_kx⁻ = reshape(W.(4Nx, 0:-1:-(Nx-1)), Nx, 1)
ω_4Ny_ky⁺ = reshape(W.(4Ny, 0:Ny-1), 1, Ny)
ω_4Ny_ky⁻ = reshape(W.(4Ny, 0:-1:-(Ny-1)), 1, Ny)

# ω_4Nx_kx⁻[1] *= 1/2
# ω_4Ny_ky⁻[1] *= 1/2

r_x_inds = [1, collect(Nx:-1:2)...]
r_y_inds = [1, collect(Ny:-1:2)...]

R4⁻⁺ = view(R4, r_x_inds, 1:Ny)
R4⁺⁻ = view(R4, 1:Nx, r_y_inds)
R4⁻⁻ = view(R4, r_x_inds, r_y_inds)

R5 = zeros(Complex{Float64}, Nx, Ny)
@. R5 = 1/4 *  ω_4Nx_kx⁻ * ω_4Ny_ky⁻ * ((R4 - R4⁻⁻) - im*(R4⁺⁻ + R4⁻⁺))

8×8 Array{Complex{Float64},2}:
       0.0+6.66134e-15im   -2.49988-0.785732im  …   0.785732+2.49988im 
   1.94814+3.43669im        2.55578-0.740972im       3.31409+0.55193im 
   4.72298+0.476895im     0.0639324+2.4974im        -1.22914-1.91031im 
  -2.40333+3.83547im       -2.39828+2.03164im       0.589639+0.237255im
   2.25379-2.25379im      -0.957065-2.19069im      -0.957065+2.19069im 
  -3.83547+2.40333im       0.589639-0.237255im  …   -2.39828-2.03164im 
 -0.476895-4.72298im       -1.22914+1.91031im      0.0639324-2.4974im  
  -3.43669-1.94814im        3.31409-0.55193im        2.55578+0.740972im

In [18]:
R6 = ifft(R5)

R7 = zeros(Nx, Ny)

p_x_inds = [1:2:Nx..., Nx:-2:2...]
p_y_inds = [1:2:Ny..., Ny:-2:2...]
R7_p = view(R7, p_x_inds, p_y_inds)

@. R7_p = real(R6)

8×8 view(::Array{Float64,2}, [1, 3, 5, 7, 8, 6, 4, 2], [1, 3, 5, 7, 8, 6, 4, 2]) with eltype Float64:
  0.330877   0.0831171  -0.192695    …   0.45892   -0.449629  -0.130942 
 -0.328204   0.475029   -0.373118       -0.368448  -0.425924   0.0930413
 -0.21389   -0.0234665   0.309153       -0.413216   0.112526  -0.232734 
 -0.450469  -0.139641   -0.00810973     -0.387948   0.345347  -0.325734 
 -0.424501   0.274832    0.406429        0.345305   0.421942   0.162678 
 -0.289294  -0.430403   -0.457426    …   0.283904  -0.357026   0.417276 
 -0.315441   0.27492     0.274299       -0.378501  -0.336134   0.046519 
  0.417586   0.375508    0.23008         0.308348  -0.224786   0.480446 

In [19]:
R7 .≈ R

8×8 BitArray{2}:
 true  true  true  true  true  true  true  true
 true  true  true  true  true  true  true  true
 true  true  true  true  true  true  true  true
 true  true  true  true  true  true  true  true
 true  true  true  true  true  true  true  true
 true  true  true  true  true  true  true  true
 true  true  true  true  true  true  true  true
 true  true  true  true  true  true  true  true